## Requirements

In [1]:
!pip install ConfigSpace

In [2]:
!pip install scikit-optimize
!pip install tune-sklearn
!pip install ConfigSpace

In [3]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedShuffleSplit, KFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from xgboost import XGBClassifier

from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import log_loss

from tune_sklearn import TuneSearchCV
from tune_sklearn import TuneGridSearchCV

import pickle

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [174]:
#word2vec
#X_10 = np.array(pickle.load(open('X_10.pkl', 'rb'))) ##done
#X_50 = np.array(pickle.load(open('X_50.pkl', 'rb')))
#X_100 = np.array(pickle.load(open('X_100.pkl', 'rb')))
#X_300 = np.array(pickle.load(open('X_300.pkl', 'rb')))
#
##doc2vec
#X_d2v_10 = pickle.load(open('X_d2v_10.pkl', 'rb'))
#X_d2v_50 = pickle.load(open('X_d2v_50.pkl', 'rb'))
#X_d2v_100 = pickle.load(open('X_d2v_100.pkl', 'rb'))
X_d2v_300 = pickle.load(open('X_d2v_300.pkl', 'rb'))

X = X_d2v_300
model_name = "X_d2v_300"

## Open the data

In [175]:
data = pd.read_excel('with_clean_texts.xlsx')
data_mask = ~pd.isna(data['Y_true'])
data = data[data_mask].copy()
data = data.reset_index(drop=True)
data['Text'] = data['Normalized_Tokens_String']

using Y_True and the name of the president

In [176]:
random_state = 12

In [177]:
data['sample_group'] = data['Party'] + "_" + data['Y_true'].astype(int).astype(str)
data['sample_group'].value_counts()

José Manuel Pureza_0                      165
Eduardo Luís Barreto Ferro Rodrigues_0    147
Edite Estrela_0                            99
António Filipe_0                           69
Fernando Negrão_0                          69
Eduardo Luís Barreto Ferro Rodrigues_1     14
José Manuel Pureza_1                       10
António Filipe_1                            7
Edite Estrela_1                             5
Fernando Negrão_1                           5
Name: sample_group, dtype: int64

In [178]:
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=random_state)
splits = kfold.split(data,data['sample_group'])

In [179]:
custom_cv = []
for train_index, test_index in splits:
     custom_cv.append((train_index, test_index))

## Brier Score Positives

In [180]:
def brier_score_positives(Y, Y_pred, threshold=0.5):
  mask = Y > threshold
  return (np.square(Y[mask] - Y_pred[mask])).mean()

## Random Forest (skipping)

In [181]:
'''
pipe = Pipeline([
       ('clf', 'passthrough'),
   ])

search_space_RF = {
                    'clf' : Categorical([RandomForestClassifier(random_state=random_state)]),
                    'clf__criterion': Categorical(['gini', 'entropy']),
                    'clf__class_weight': Categorical(['balanced', 'balanced_subsample', None]),
                    'clf__n_estimators': Integer(10, 1000, prior='uniform')}

my_scores =   { 'accuracy' :make_scorer(accuracy_score),
                'recall'   :make_scorer(recall_score),
                'precision':make_scorer(precision_score),
                'f1'       :make_scorer(fbeta_score, beta = 1),
                'roc_auc'  :make_scorer(roc_auc_score), 
                'log_loss' :make_scorer(log_loss, needs_proba=True),
                'brier_loss': make_scorer(brier_score_loss, needs_proba=True), 
                'brier_positives':make_scorer(brier_score_positives, needs_proba=True)}

optmizer1 = TuneSearchCV(
    estimator=pipe,
    param_distributions=search_space_RF,
    search_optimization="bayesian",
    #early_stopping=True,
    max_iters=1000,
    #pipeline_auto_early_stop=True,
    scoring=my_scores,
    refit='f1',
    cv=custom_cv,
    n_trials=50,
    verbose=1,
    random_state=random_state,
    use_gpu=True)
'''

'\npipe = Pipeline([\n       (\'clf\', \'passthrough\'),\n   ])\n\nsearch_space_RF = {\n                    \'clf\' : Categorical([RandomForestClassifier(random_state=random_state)]),\n                    \'clf__criterion\': Categorical([\'gini\', \'entropy\']),\n                    \'clf__class_weight\': Categorical([\'balanced\', \'balanced_subsample\', None]),\n                    \'clf__n_estimators\': Integer(10, 1000, prior=\'uniform\')}\n\nmy_scores =   { \'accuracy\' :make_scorer(accuracy_score),\n                \'recall\'   :make_scorer(recall_score),\n                \'precision\':make_scorer(precision_score),\n                \'f1\'       :make_scorer(fbeta_score, beta = 1),\n                \'roc_auc\'  :make_scorer(roc_auc_score), \n                \'log_loss\' :make_scorer(log_loss, needs_proba=True),\n                \'brier_loss\': make_scorer(brier_score_loss, needs_proba=True), \n                \'brier_positives\':make_scorer(brier_score_positives, needs_proba=True)

In [182]:
'''
optmizer1.fit(X, data['Y_true'].to_numpy()),
Results_RF = pd.DataFrame(optmizer1.cv_results_).sort_values(by=['rank_test_f1'])
Results_RF.to_excel(model_name + '_Results_RF.xlsx')
'''

"\noptmizer1.fit(X, data['Y_true'].to_numpy()),\nResults_RF = pd.DataFrame(optmizer1.cv_results_).sort_values(by=['rank_test_f1'])\nResults_RF.to_excel(model_name + '_Results_RF.xlsx')\n"

## SVC (skipping)

In [183]:
'''
pipe = Pipeline([
       ('clf', 'passthrough'),
   ])

search_space_SVC = {
                    'clf' : Categorical([SVC(max_iter=10000, random_state=random_state, probability=True)]),
                    'clf__kernel' : Categorical(['linear', 'poly', 'rbf', 'sigmoid']),
                    'clf__C': Real(1e-6, 1e+6, prior='log-uniform')}

# my_scores =   { 'accuracy' :make_scorer(accuracy_score),
#                 'recall'   :make_scorer(recall_score),
#                 'precision':make_scorer(precision_score),
#                 'f1'       :make_scorer(fbeta_score, beta = 1),
#                 'roc_auc'  :make_scorer(roc_auc_score), 
#                 'log_loss' :make_scorer(log_loss),
#                 'brier_loss': make_scorer(brier_score_loss)}

optmizer2 = TuneSearchCV(
    estimator=pipe,
    param_distributions=search_space_SVC,
    search_optimization="bayesian",
    #early_stopping=True,
    #pipeline_auto_early_stop=True,
    scoring=my_scores,
    refit='f1',
    cv=custom_cv,
    n_trials=100,
    verbose=1,
    random_state=random_state,
    use_gpu=True)
'''

'\npipe = Pipeline([\n       (\'clf\', \'passthrough\'),\n   ])\n\nsearch_space_SVC = {\n                    \'clf\' : Categorical([SVC(max_iter=10000, random_state=random_state, probability=True)]),\n                    \'clf__kernel\' : Categorical([\'linear\', \'poly\', \'rbf\', \'sigmoid\']),\n                    \'clf__C\': Real(1e-6, 1e+6, prior=\'log-uniform\')}\n\n# my_scores =   { \'accuracy\' :make_scorer(accuracy_score),\n#                 \'recall\'   :make_scorer(recall_score),\n#                 \'precision\':make_scorer(precision_score),\n#                 \'f1\'       :make_scorer(fbeta_score, beta = 1),\n#                 \'roc_auc\'  :make_scorer(roc_auc_score), \n#                 \'log_loss\' :make_scorer(log_loss),\n#                 \'brier_loss\': make_scorer(brier_score_loss)}\n\noptmizer2 = TuneSearchCV(\n    estimator=pipe,\n    param_distributions=search_space_SVC,\n    search_optimization="bayesian",\n    #early_stopping=True,\n    #pipeline_auto_early_stop=

In [184]:
'''
optmizer2.fit(X, data['Y_true'].to_numpy()),
Results_SVC = pd.DataFrame(optmizer2.cv_results_).sort_values(by=['rank_test_f1'])
Results_SVC.to_excel(model_name + '_Results_SVC.xlsx')
'''

"\noptmizer2.fit(X, data['Y_true'].to_numpy()),\nResults_SVC = pd.DataFrame(optmizer2.cv_results_).sort_values(by=['rank_test_f1'])\nResults_SVC.to_excel(model_name + '_Results_SVC.xlsx')\n"

# Naive Bayes (error)

In [185]:
'''
pipe = Pipeline([
       ('step3', SelectKBest(ANOVA, k=100))
       ('clf', 'passthrough'),
   ])

search_space_bayes = {#'step1' : Categorical([CountVectorizer(), CountVectorizer(ngram_range=(1, 2)), CountVectorizer(ngram_range=(1, 3))]),
                      'clf' : Categorical([naive_bayes.MultinomialNB(), naive_bayes.ComplementNB(), naive_bayes.BernoulliNB(binarize=0.1)]),
                      'clf__alpha': Real(1e-4, 10, prior='uniform')}

# my_scores =   { 'accuracy' :make_scorer(accuracy_score),
#                 'recall'   :make_scorer(recall_score),
#                 'precision':make_scorer(precision_score),
#                 'f1'       :make_scorer(fbeta_score, beta = 1),
#                 'roc_auc'  :make_scorer(roc_auc_score), 
#                 'log_loss' :make_scorer(log_loss),
#                 'brier_loss': make_scorer(brier_score_loss)}

optmizer4 = TuneSearchCV(
    estimator=pipe,
    param_distributions=search_space_bayes,
    search_optimization="bayesian",
    #early_stopping=True,
    #pipeline_auto_early_stop=True,
    scoring=my_scores,
    refit='f1',
    cv=custom_cv,
    n_trials=50,
    verbose=1,
    random_state=random_state,
    use_gpu=True)
'''

'\npipe = Pipeline([\n       (\'step3\', SelectKBest(ANOVA, k=100))\n       (\'clf\', \'passthrough\'),\n   ])\n\nsearch_space_bayes = {#\'step1\' : Categorical([CountVectorizer(), CountVectorizer(ngram_range=(1, 2)), CountVectorizer(ngram_range=(1, 3))]),\n                      \'clf\' : Categorical([naive_bayes.MultinomialNB(), naive_bayes.ComplementNB(), naive_bayes.BernoulliNB(binarize=0.1)]),\n                      \'clf__alpha\': Real(1e-4, 10, prior=\'uniform\')}\n\n# my_scores =   { \'accuracy\' :make_scorer(accuracy_score),\n#                 \'recall\'   :make_scorer(recall_score),\n#                 \'precision\':make_scorer(precision_score),\n#                 \'f1\'       :make_scorer(fbeta_score, beta = 1),\n#                 \'roc_auc\'  :make_scorer(roc_auc_score), \n#                 \'log_loss\' :make_scorer(log_loss),\n#                 \'brier_loss\': make_scorer(brier_score_loss)}\n\noptmizer4 = TuneSearchCV(\n    estimator=pipe,\n    param_distributions=search_spa

In [186]:
'''
models = {
    'X_10' :      optmizer3.fit(X_10, data['Y_true'].to_numpy()),
    'X_50' :      optmizer3.fit(X_50, data['Y_true'].to_numpy()),
    'X_100' :     optmizer3.fit(X_100, data['Y_true'].to_numpy()),
    'X_300' :     optmizer3.fit(X_300, data['Y_true'].to_numpy()),
    'X_d2v_10' :  optmizer3.fit( X_d2v_10, data['Y_true'].to_numpy()),
    'X_d2v_50' :  optmizer3.fit( X_d2v_50, data['Y_true'].to_numpy()),
    'X_d2v_100' : optmizer3.fit(X_d2v_100, data['Y_true'].to_numpy()),
    'X_d2v_300' : optmizer3.fit(X_d2v_300, data['Y_true'].to_numpy()),
}

for model in models:
    Results_RF = pd.DataFrame(models[model].cv_results_).sort_values(by=['rank_test_f1'])
    Results_RF.to_excel(model + '_Results_Bayes.xlsx')
'''

"\nmodels = {\n    'X_10' :      optmizer3.fit(X_10, data['Y_true'].to_numpy()),\n    'X_50' :      optmizer3.fit(X_50, data['Y_true'].to_numpy()),\n    'X_100' :     optmizer3.fit(X_100, data['Y_true'].to_numpy()),\n    'X_300' :     optmizer3.fit(X_300, data['Y_true'].to_numpy()),\n    'X_d2v_10' :  optmizer3.fit( X_d2v_10, data['Y_true'].to_numpy()),\n    'X_d2v_50' :  optmizer3.fit( X_d2v_50, data['Y_true'].to_numpy()),\n    'X_d2v_100' : optmizer3.fit(X_d2v_100, data['Y_true'].to_numpy()),\n    'X_d2v_300' : optmizer3.fit(X_d2v_300, data['Y_true'].to_numpy()),\n}\n\nfor model in models:\n    Results_RF = pd.DataFrame(models[model].cv_results_).sort_values(by=['rank_test_f1'])\n    Results_RF.to_excel(model + '_Results_Bayes.xlsx')\n"

## Naive Bayes TFIDF (skipping)

In [187]:
'''
pipe = Pipeline([
       ('step1', CountVectorizer()),
       ('step2', TfidfTransformer()),
       ('clf', 'passthrough'),
   ])

search_space_bayes = {#'step1' : Categorical([CountVectorizer(), CountVectorizer(ngram_range=(1, 2)), CountVectorizer(ngram_range=(1, 3))]),
                      'clf' : Categorical([naive_bayes.MultinomialNB(), naive_bayes.ComplementNB()]),
                      'clf__alpha': Real(1e-4, 10, prior='uniform')}

# my_scores =   { 'accuracy' :make_scorer(accuracy_score),
#                 'recall'   :make_scorer(recall_score),
#                 'precision':make_scorer(precision_score),
#                 'f1'       :make_scorer(fbeta_score, beta = 1),
#                 'roc_auc'  :make_scorer(roc_auc_score), 
#                 'log_loss' :make_scorer(log_loss),
#                 'brier_loss': make_scorer(brier_score_loss)}

optmizer10 = TuneSearchCV(
    estimator=pipe,
    param_distributions=search_space_bayes,
    search_optimization="bayesian",
    #early_stopping=True,
    #pipeline_auto_early_stop=True,
    scoring=my_scores,
    refit='f1',
    cv=custom_cv,
    n_trials=50,
    verbose=1,
    random_state=random_state,
    use_gpu=True)
optmizer10.fit(data['Text'].to_numpy(), data['Y_true'].to_numpy())
'''

'\npipe = Pipeline([\n       (\'step1\', CountVectorizer()),\n       (\'step2\', TfidfTransformer()),\n       (\'clf\', \'passthrough\'),\n   ])\n\nsearch_space_bayes = {#\'step1\' : Categorical([CountVectorizer(), CountVectorizer(ngram_range=(1, 2)), CountVectorizer(ngram_range=(1, 3))]),\n                      \'clf\' : Categorical([naive_bayes.MultinomialNB(), naive_bayes.ComplementNB()]),\n                      \'clf__alpha\': Real(1e-4, 10, prior=\'uniform\')}\n\n# my_scores =   { \'accuracy\' :make_scorer(accuracy_score),\n#                 \'recall\'   :make_scorer(recall_score),\n#                 \'precision\':make_scorer(precision_score),\n#                 \'f1\'       :make_scorer(fbeta_score, beta = 1),\n#                 \'roc_auc\'  :make_scorer(roc_auc_score), \n#                 \'log_loss\' :make_scorer(log_loss),\n#                 \'brier_loss\': make_scorer(brier_score_loss)}\n\noptmizer10 = TuneSearchCV(\n    estimator=pipe,\n    param_distributions=search_space_b

In [188]:
'''
Results_bayes2 = pd.DataFrame(optmizer10.cv_results_).sort_values(by=['rank_test_f1'])
Results_bayes2.to_excel("Normalized_Results_bayes_TFID.xlsx")
Results_bayes2
'''

'\nResults_bayes2 = pd.DataFrame(optmizer10.cv_results_).sort_values(by=[\'rank_test_f1\'])\nResults_bayes2.to_excel("Normalized_Results_bayes_TFID.xlsx")\nResults_bayes2\n'

# Logistic Regression 1

In [189]:
pipe = Pipeline([
       ('clf', 'passthrough'),
   ])

search_space_LR1 = {
                    'clf' : Categorical([LogisticRegression(random_state=random_state, max_iter=1000)]),
                    'clf__C': Real(1e-6, 1e+6, prior='log-uniform'),
                    'clf__class_weight': Categorical(['balanced', None]),
                    'clf__penalty': Categorical(['l2']),
                    'clf__solver': Categorical(['newton-cg', 'lbfgs', 'sag', 'saga'])}

my_scores =   { 'accuracy' :make_scorer(accuracy_score),
                'recall'   :make_scorer(recall_score),
                'precision':make_scorer(precision_score),
                'f1'       :make_scorer(fbeta_score, beta = 1),
                'roc_auc'  :make_scorer(roc_auc_score), 
                'log_loss' :make_scorer(log_loss),
                'brier_loss': make_scorer(brier_score_loss)}

optmizer5 = TuneSearchCV(
    estimator=pipe,
    param_distributions=search_space_LR1,
    search_optimization="bayesian",
    #early_stopping=True,
    #pipeline_auto_early_stop=True,
    scoring=my_scores,
    refit='f1',
    cv=custom_cv,
    n_trials=100,
    verbose=1,
    random_state=random_state,
    use_gpu=True)

In [190]:
optmizer5.fit(X, data['Y_true'].to_numpy()),
Results_LR1 = pd.DataFrame(optmizer5.cv_results_).sort_values(by=['rank_test_f1'])
Results_LR1.to_excel(model_name + '_Results_LR1.xlsx')

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [191]:
#Limited-memory Broyden–Fletcher–Goldfarb–Shanno Algorithm
#If I dare say that when dataset is small, L-BFGS relatively performs the best compared to other methods especially it saves a lot of memory, however there are some “serious” drawbacks such that if it is unsafeguarded, it may not converge to anything.

## Logistic Regression 2

In [192]:
pipe = Pipeline([
       ('clf', 'passthrough'),
   ])

search_space_LR2 = {
                    'clf' : Categorical([LogisticRegression(random_state=random_state, max_iter=1000)]),
                    'clf__C': Real(1e-6, 1e+6, prior='log-uniform'),
                    'clf__class_weight': Categorical(['balanced', None]),
                    'clf__penalty': Categorical(['l1']),
                    'clf__solver': Categorical(['liblinear', 'saga'])}

# my_scores =   { 'accuracy' :make_scorer(accuracy_score),
#                 'recall'   :make_scorer(recall_score),
#                 'precision':make_scorer(precision_score),
#                 'f1'       :make_scorer(fbeta_score, beta = 1),
#                 'roc_auc'  :make_scorer(roc_auc_score), 
#                 'log_loss' :make_scorer(log_loss),
#                 'brier_loss': make_scorer(brier_score_loss)}

optmizer6 = TuneSearchCV(
    estimator=pipe,
    param_distributions=search_space_LR2,
    search_optimization="bayesian",
    #early_stopping=True,
    #pipeline_auto_early_stop=True,
    scoring=my_scores,
    refit='f1',
    cv=custom_cv,
    n_trials=50,
    verbose=1,
    random_state=random_state,
    use_gpu=True)

In [193]:
optmizer6.fit(X, data['Y_true'].to_numpy()),
Results_LR2 = pd.DataFrame(optmizer6.cv_results_).sort_values(by=['rank_test_f1'])
Results_LR2.to_excel(model_name + '_Results_LR2.xlsx')

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


# kNN


In [194]:
pipe = Pipeline([
       ('clf', 'passthrough'),
   ])

search_space_kNN = {
                    'clf' : Categorical([KNeighborsClassifier()]),
                    'clf__weights' : Categorical(['uniform', 'distance']),
                    'clf__n_neighbors': Integer(5, 15, prior='uniform')}

# my_scores =   { 'accuracy' :make_scorer(accuracy_score),
#                 'recall'   :make_scorer(recall_score),
#                 'precision':make_scorer(precision_score),
#                 'f1'       :make_scorer(fbeta_score, beta = 1),
#                 'roc_auc'  :make_scorer(roc_auc_score), 
#                 'log_loss' :make_scorer(log_loss),
#                 'brier_loss': make_scorer(brier_score_loss)}

optmizer7 = TuneSearchCV(
    estimator=pipe,
    param_distributions=search_space_kNN,
    search_optimization="bayesian",
    #early_stopping=True,
    #pipeline_auto_early_stop=True,
    scoring=my_scores,
    refit='f1',
    cv=custom_cv,
    n_trials=100,
    verbose=1,
    random_state=random_state,
    use_gpu=True)

In [195]:
optmizer7.fit(X, data['Y_true'].to_numpy()),
Results_KNN = pd.DataFrame(optmizer7.cv_results_).sort_values(by=['rank_test_f1'])
Results_KNN.to_excel(model_name + '_Results_KNN.xlsx')

# XGBoost

In [196]:
pipe = Pipeline([
       ('clf', 'passthrough'),
   ])

search_space_XGBoost = {
                        'clf' : Categorical([XGBClassifier(seed=random_state)]),
                        'clf__booster' : Categorical(['gbtree', 'gblinear', 'dart']),
                        'clf__subsample' : Real(0.1, 1, prior='uniform'),
                        'clf__sampling_method' : Categorical(['uniform', 'gradient_based']),
                        'clf__learning_rate' : Real(1e-6, 0.3, prior='uniform')}

my_scores =   { 'accuracy' :make_scorer(accuracy_score),
                 'recall'   :make_scorer(recall_score),
                 'precision':make_scorer(precision_score),
                 'f1'       :make_scorer(fbeta_score, beta = 1),
                 'roc_auc'  :make_scorer(roc_auc_score), 
                 'log_loss' :make_scorer(log_loss),
                 'brier_loss': make_scorer(brier_score_loss)}

optmizer8 = TuneSearchCV(
    estimator=pipe,
    param_distributions=search_space_XGBoost,
    search_optimization="bayesian",
    #early_stopping=True,
    #pipeline_auto_early_stop=True,
    scoring=my_scores,
    refit='f1',
    cv=custom_cv,
    n_trials=100,
    verbose=1,
    random_state=random_state,
    use_gpu=True)

In [197]:
optmizer8.fit(X, data['Y_true'].to_numpy()),
Results_XGBoost = pd.DataFrame(optmizer8.cv_results_).sort_values(by=['rank_test_f1'])
Results_XGBoost.to_excel(model_name + '_Results_XGBoost.xlsx')

# SGD Classifier (skipping)


In [198]:
'''
pipe = Pipeline([
       ('clf', 'passthrough'),
   ])

search_space_SGD = {
                    'clf' : Categorical([SGDClassifier(random_state=random_state)]),
                    'clf__loss' : Categorical(['log', 'modified_huber']),
                    'clf__penalty':  Categorical(['l2', 'l1', 'elasticnet']),
                    'clf__alpha': Real(1e-6, 1e+6, prior='log-uniform')}

# my_scores =   { 'accuracy' :make_scorer(accuracy_score),
#                 'recall'   :make_scorer(recall_score),
#                 'precision':make_scorer(precision_score),
#                 'f1'       :make_scorer(fbeta_score, beta = 1),
#                 'roc_auc'  :make_scorer(roc_auc_score), 
#                 'log_loss' :make_scorer(log_loss),
#                 'brier_loss': make_scorer(brier_score_loss)}

optmizer9 = TuneSearchCV(
    estimator=pipe,
    param_distributions=search_space_SGD,
    search_optimization="bayesian",
    #early_stopping=True,
    #pipeline_auto_early_stop=True,
    scoring=my_scores,
    refit='f1',
    cv=custom_cv,
    n_trials=100,
    verbose=1,
    random_state=random_state,
    use_gpu=True)
'''

'\npipe = Pipeline([\n       (\'clf\', \'passthrough\'),\n   ])\n\nsearch_space_SGD = {\n                    \'clf\' : Categorical([SGDClassifier(random_state=random_state)]),\n                    \'clf__loss\' : Categorical([\'log\', \'modified_huber\']),\n                    \'clf__penalty\':  Categorical([\'l2\', \'l1\', \'elasticnet\']),\n                    \'clf__alpha\': Real(1e-6, 1e+6, prior=\'log-uniform\')}\n\n# my_scores =   { \'accuracy\' :make_scorer(accuracy_score),\n#                 \'recall\'   :make_scorer(recall_score),\n#                 \'precision\':make_scorer(precision_score),\n#                 \'f1\'       :make_scorer(fbeta_score, beta = 1),\n#                 \'roc_auc\'  :make_scorer(roc_auc_score), \n#                 \'log_loss\' :make_scorer(log_loss),\n#                 \'brier_loss\': make_scorer(brier_score_loss)}\n\noptmizer9 = TuneSearchCV(\n    estimator=pipe,\n    param_distributions=search_space_SGD,\n    search_optimization="bayesian",\n    #ea

In [199]:
'''
optmizer9.fit(X, data['Y_true'].to_numpy()),
Results_SGD = pd.DataFrame(optmizer9.cv_results_).sort_values(by=['rank_test_f1'])
Results_SGD.to_excel(model_name + '_Results_SGD.xlsx')
'''

"\noptmizer9.fit(X, data['Y_true'].to_numpy()),\nResults_SGD = pd.DataFrame(optmizer9.cv_results_).sort_values(by=['rank_test_f1'])\nResults_SGD.to_excel(model_name + '_Results_SGD.xlsx')\n"

# BERTimbau (skipping)

In [200]:

'''
from transformers import AutoConfig, AutoTokenizer, TFAutoModel

class BERTimbauTokenizer(object):

  def __init__(self, max_len=3000):
    self.Tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
    self.max_len = max_len

  def tokenize(self, sentence):
      tokens = Tokenizer.encode_plus(sentence, max_length=self.max_len,
                                    truncation=True, padding='max_length',
                                    add_special_tokens=True, return_attention_mask=True,
                                    return_token_type_ids=False, return_tensors='tf')
      return np.array(tf.reshape(tokens['input_ids'], [self.max_len])), np.array(tf.reshape(tokens['attention_mask'], [self.max_len]))

  def token_encode(self, X):
    input_ids = []
    masks = []
    for text in X:
      text_input_id, text_mask = tokenize(text)
      input_ids.append(tf.reshape(text_input_id, [self.max_len]))
      masks.append(tf.reshape(text_mask, [self.max_len]))
    return np.array(input_ids), np.array(masks)
  
  def fit(self, X=None, Y=None):
    return

  def transform(self, X):
    return self.token_encode(X)
'''

"\nfrom transformers import AutoConfig, AutoTokenizer, TFAutoModel\n\nclass BERTimbauTokenizer(object):\n\n  def __init__(self, max_len=3000):\n    self.Tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')\n    self.max_len = max_len\n\n  def tokenize(self, sentence):\n      tokens = Tokenizer.encode_plus(sentence, max_length=self.max_len,\n                                    truncation=True, padding='max_length',\n                                    add_special_tokens=True, return_attention_mask=True,\n                                    return_token_type_ids=False, return_tensors='tf')\n      return np.array(tf.reshape(tokens['input_ids'], [self.max_len])), np.array(tf.reshape(tokens['attention_mask'], [self.max_len]))\n\n  def token_encode(self, X):\n    input_ids = []\n    masks = []\n    for text in X:\n      text_input_id, text_mask = tokenize(text)\n      input_ids.append(tf.reshape(text_input_id, [self.max_len]))\n      masks.append(tf.reshape(tex

In [201]:
'''
def build_model(max_len=512, dropout=True, num_layers_ftuning=2):
  input_ids = tf.keras.layers.Input(shape=(max_len,), name='input_ids', dtype=tf.int32)
  mask = tf.keras.layers.Input(shape=(max_len,), name='attention_mask', dtype=tf.int32)
  
  # By December 12th Hugging Face's Transformer Library only had the PyTorch version of BERTimbau so we needed to
  # convert the PyTorch model to TensorFlow using the AutoConfig class and adding from_pt (from pytorch) equal to True
  config = AutoConfig.from_pretrained('neuralmind/bert-large-portuguese-cased')
  BERTimbau = TFAutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased', from_pt=True, config=config)

  last_bert_hidden_layer = BERTimbau(input_ids, attention_mask=mask)[0]

  net = tf.keras.layers.Dense(64, activation='relu')(last_bert_hidden_layer)
  if dropout:
    net = tf.keras.layers.Dropout(0.2)(net)
  net = tf.keras.layers.Flatten()(net)
  if num_layers_ftuning>1:
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    if dropout:
      net = tf.keras.layers.Dropout(0.2)(net)
  out = tf.keras.layers.Dense(1, activation='sigmoid')(net)

  model = tf.keras.models.Model(inputs=[input_ids, mask], outputs=out)
  model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy', tfa.metrics.F1Score(num_classes=3, average="micro", threshold=0.5)])

  return model
'''

'\ndef build_model(max_len=512, dropout=True, num_layers_ftuning=2):\n  input_ids = tf.keras.layers.Input(shape=(max_len,), name=\'input_ids\', dtype=tf.int32)\n  mask = tf.keras.layers.Input(shape=(max_len,), name=\'attention_mask\', dtype=tf.int32)\n  \n  # By December 12th Hugging Face\'s Transformer Library only had the PyTorch version of BERTimbau so we needed to\n  # convert the PyTorch model to TensorFlow using the AutoConfig class and adding from_pt (from pytorch) equal to True\n  config = AutoConfig.from_pretrained(\'neuralmind/bert-large-portuguese-cased\')\n  BERTimbau = TFAutoModel.from_pretrained(\'neuralmind/bert-large-portuguese-cased\', from_pt=True, config=config)\n\n  last_bert_hidden_layer = BERTimbau(input_ids, attention_mask=mask)[0]\n\n  net = tf.keras.layers.Dense(64, activation=\'relu\')(last_bert_hidden_layer)\n  if dropout:\n    net = tf.keras.layers.Dropout(0.2)(net)\n  net = tf.keras.layers.Flatten()(net)\n  if num_layers_ftuning>1:\n    net = tf.keras.layer